In [39]:
import pandas as pd
import plotly.express as px
import numpy as np
from ydata_profiling import ProfileReport
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans



In [9]:
a4_data = pd.read_csv("./dataset/2023-02-08-DATA624-Assignment4-Data.csv")
a4_data.describe()
a4_data.head
# a4_data.columns

<bound method NDFrame.head of              0         1         2         3         4         5         6  \
0    -1.606951  0.873530  0.354386 -0.376583 -0.123913 -5.574885  0.115763   
1     0.704287 -1.828743 -0.116858  0.147165 -0.296007 -5.944087 -0.156580   
2     1.424599 -0.281860  0.841544 -0.655310  0.378469 -6.604756 -0.426063   
3     1.390018  0.289805  1.093028 -0.690624 -0.548168 -5.948213 -0.081209   
4     1.032563 -1.042404  1.019796  0.888428 -0.102096 -5.970099 -0.071962   
...        ...       ...       ...       ...       ...       ...       ...   
6165 -3.907976  2.522889 -2.677499  1.191893 -1.437921 -1.749991 -0.189743   
6166  3.330155 -4.534200  1.625227 -0.485558  0.914868 -0.527034 -0.507081   
6167 -3.300942 -2.333507 -1.022910 -2.212949  2.440453  2.567430 -0.558333   
6168  5.778613  0.536104  0.906734 -0.986751 -0.447457 -0.161881 -0.292931   
6169 -5.966899  0.599078  0.014040 -0.108468  0.145487  0.048156  0.036127   

             7         8         

##### Use Pandas Profiling to skim the dataset for data cleaning and column selections

In [7]:
profile = ProfileReport(
    a4_data, 
    title="Nifty Profiling Report",
    # dark_mode=True,
    orange_mode=True,
    minimal=True,
)

In [8]:
profile.to_file("profile_a4.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

##### Based on the result of profiling above, 3.1 column will be removed due to all constant zero value (will see how different it will be) 4.1 column will be keeped in the table, it might represent the target value.

#####  Use KMeans to get general idea of estiamted number of clusters, then apply other algorithms based on the range of K. Create blobs, circle and moons shape for each one test with three following algorithms and see which one has the best performances.

#### To discover how many clusters in the dataset (check all pairs), will use Agglomerative clustering (for detecting Large n_samples and n_clusters) and OPTICS (for detecting Large n_clusters and n_samples also for Large dataset, outlier removal, data reduction). The last one will use t-Distributed Stochastic Neighbor Embedding (t-SNE) to see how data separated into clusters. 

In [36]:
df = a4_data
# df = circles
# df = moons
sscore = []
# starting with 2 clusters now becuase the silhouette isn't defined for 1
for i in range(2,10):
    kmeans = KMeans(
        n_clusters = i, # Number of clusters to find
        init = "k-means++", # How to place the initial cluster centroids,
        max_iter= 100, # Maximum number of iterations for the algorithm to run
        tol=0.0001, # Roughly how much the centroids need to change between iterations to keep going
    ).fit(
        df
    )
    
    silhouette_avg = silhouette_score(df, kmeans.labels_)
    sscore.append(silhouette_avg)
    

px.line(y = sscore,x = range(2,10), markers=True)

In [60]:
a4_data = pd.DataFrame(a4_data)
a4_test = a4_data.iloc[:, :9]
a4_test.columns=['column1','column2','column3','column4','column5','column6','column7','column8','column9']
# print(a4_test)
test = a4_test[['column1','column2']].values.tolist()
# from sklearn.datasets import make_blobs
# blobs = make_blobs(
#     n_samples=300,
#     n_features=2,
#     centers=3,
#     # return_centers=True,
# )
# df_concatenated = pd.concat([df, pd.DataFrame(np_array)], axis=1)
print(test[0])
blobs = pd.concat([
    pd.DataFrame(test[0]),
    pd.DataFrame(test[1],columns=['labels']).astype('category'),
],axis=1)
blobs.columns = [str(x) for x in blobs.columns]
blobs.head()

[-1.606951114175826, 0.8735295569519647]


,0,labels
0,-1.606951,0.704287
1,0.873530,-1.828743


In [59]:
from sklearn.cluster import AgglomerativeClustering
test = blobs
clusters = AgglomerativeClustering(
    n_clusters = 2, # Number of clusters to find
    # linkage="ward", #default
    linkage="single",
).fit(
    test
)
clusters.labels_[::10]

array([1])

In [50]:
(pd.concat([
    test,
    pd.DataFrame(
        clusters.labels_,
        columns=['cluster']
    ).astype('category')
], axis=1)
 .plot.scatter(
     x='0',
     y='1',
     color='cluster',
     symbol='labels',
 )
)

KeyError: '1'

In [37]:
from sklearn.manifold import TSNE

In [38]:
ts = TSNE(
    perplexity=100, # Roughly the "size" of the clusters to look for (original paper
                   # recommends in the 5-50 range, but in general should be less than
                   # then number of points in your dataset
    learning_rate="auto",
    n_iter=2000,
    init='pca',
).fit_transform(a4_data)

(pd.concat([
    pd.DataFrame(ts),
    a4_data['labels'],
], axis=1)
 .plot.scatter(
     x=0,
     y=1,
     color='labels',
 )
)

/Users/sun/opt/anaconda3/envs/dataenv/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:996: FutureWarning:

The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.



KeyError: 'labels'